#### Candy Crush in Python - Extras 3 - Game Over

This file shows the code to check when the game is over.

## Current Game Code

In [1]:
import numpy as np
from gamegrid import GameGrid
from IPython.display import display
from ipywidgets import Button, HBox, Label, VBox
import time


def show_board(board_array):
    grid = GameGrid()
    grid.data = board_array
    display(grid)
    

def swap_items(board, first, second):    
    dist = np.sum(np.abs(np.subtract(second, first)))
    
    is_beside = dist <= 1
    
    if(is_beside):
        tmp = board[first]
        board[first] = board[second]        
        board[second] = tmp 


def horiz_match_length(board, point):
    first = board[point]
    (r, c) = point
    (row_count, col_count) = board.shape
    same_tile_count = 1
    
    for i in range(c+1, col_count):
        if(board[r, i] != first):
            return same_tile_count
        else:
            same_tile_count = same_tile_count + 1
            
    return same_tile_count


def vert_match_length(board, point):
    first = board[point]
    (r,c) = point
    (row_count, col_count) = board.shape
    same_tile_count = 1    
    
    for i in range(r+1, row_count):
        if(board[i,c] != first):
            return same_tile_count
        else:
            same_tile_count = same_tile_count + 1
            
    return same_tile_count


def find_matches(board, required_matches):
    horiz_matches = []
    vert_matches = []
    
    (row_count, col_count) = board.shape
    
    for r in range(row_count):
        for c in range(col_count):
            point = (r,c)
            
            horiz_len = horiz_match_length(board, point)
            if horiz_len >= required_matches:
                horiz_matches.append((r, c, horiz_len))  
            
            vert_len = vert_match_length(board, point)
            if vert_len >= required_matches:
                vert_matches.append((r, c, vert_len))
                
    return (horiz_matches, vert_matches)


def remove_matching_tiles(board, horiz_matches, vert_matches):
    for (r, c, length) in horiz_matches:
         for i in range(c, c+length):
                board[r, i] = -1
    
    for (r, c, length) in vert_matches:
        for i in range(r, r+length):
            board[i, c] = -1
            
            
def pull_down_cell(board, point, number_of_types):
    (r, c) = point
    for r_above in reversed(range(r)):    
        if(board[r_above, c] != -1):            
            board[r,c] = board[r_above,c]
            board[r_above,c] = -1
            break
    if(board[r,c] == -1):
        board[r,c] = np.random.randint(number_of_types)
        
        
def pull_down_cells(board, number_of_types):
    for r in reversed(range(board.shape[0])):
        for c in range(board.shape[1]):
            if(board[r,c] == -1):
                pull_down_cell(board, (r, c), number_of_types)
                

def update_grid(grid, board):
    grid.data = board
    time.sleep(0.25)
    
                
def process_matches(board, number_of_types, required_matches, refresh):
    
    # Pass in the required_matches
    (h_matches, v_matches) = find_matches(board, required_matches)
    remove_matching_tiles(board, h_matches, v_matches)
    match_count = len(h_matches) + len(v_matches)    
    
    if match_count > 0:        
        refresh()
        pull_down_cells(board, number_of_types)
        refresh()
        # Pass the required_matches to the next process_matches
        count = process_matches(board, number_of_types, required_matches, refresh)
        match_count = match_count + count
        
    return match_count


class ScoreControl:
    
    _score = 0
    
    def __init__(self, label):
        self._label = label
        self._update_label()            
        
    @property
    def label(self):
        return self._label
        
    @property
    def score(self):
        return self._score
    
    @score.setter
    def score(self, score):
        self._score = score
        self._update_label()
        
    def add_to_score(self, score):
        self.score = self._score + score
        
        
    def _update_label(self):
        self._label.value = "Score: " + str(self._score)
    

def handle_click(grid, point, required_matches, handle_score):   
        
    grid.toggle_select(point)
    
    if len(grid.selected) < 2:
        return    
    
    board = grid.data
    refresh = lambda: update_grid(grid, board)
    last_point = grid.selected[0]
   
    swap_items(board, last_point, point)
    refresh()
    
    grid.toggle_select(last_point)
    grid.toggle_select(point)
    
    number_of_types = len(grid.images)
    match_count = process_matches(board, number_of_types, required_matches, refresh)
    handle_score(match_count)
    
    if(match_count == 0):
        # We havent found any matches so swap back
        swap_items(board, last_point, point)      
        refresh()


def create_grid(required_matches, handle_score):        
    # Create a new grid and 
    grid = GameGrid()    
    grid.on_click(
        lambda g, r, c: handle_click(g, (r, c), required_matches, handle_score))        
    return grid


def init_grid(grid, rows, cols, required_matches):    
    number_of_types = len(grid.images)
    board = np.random.randint(number_of_types, size=(rows, cols))    
    process_matches(board, number_of_types, required_matches, lambda: None)
    grid.data = board    
    

def new_game(rows, cols, required_matches):
    
    score_label = Label()
    score = ScoreControl(score_label)
    
    grid = create_grid(required_matches, lambda m: score.add_to_score(m*10))
    init_grid(grid, rows, cols, required_matches)
    
    new_game_button = Button(description="New Game")
    control_panel = VBox([new_game_button, score_label])
    game = HBox([grid, control_panel])
    new_game_button.on_click(lambda b: init_grid(grid, rows, cols, required_matches))
    return game

## Game Over

In [2]:
def test_swap(board, point1, point2, required_matches):    
    swap_items(board, point1, point2)
    (h, v) = find_matches(board, required_matches)
    count = len(h) + len(v)
    swap_items(board, point1, point2)
    return count > 0

def test_swaps(board, r, c, required_matches):    
    matches = []
    point = (r, c)
    right = (r, c + 1)
    down = (r + 1, c)
    
    if c < board.shape[1] - 1 and test_swap(board, point, right, required_matches):
        matches.append((point, right))
    if r < board.shape[0] - 1 and test_swap(board, point, down, required_matches):
        matches.append((point, down))
    
    return matches

def test_all(board, required_matches):   
    all_matches = []
    for r in range(0, board.shape[0]):
        for c in range(0, board.shape[1]):
            matches = test_swaps(board, r, c, required_matches)
            all_matches = all_matches + matches            
        
    return all_matches


def handle_click(grid, point, required_matches, handle_score, handle_game_over):   
        
    grid.toggle_select(point)
    
    if len(grid.selected) < 2:
        return    
    
    board = grid.data
    refresh = lambda: update_grid(grid, board)
    last_point = grid.selected[0]
   
    swap_items(board, last_point, point)
    refresh()
    
    grid.toggle_select(last_point)
    grid.toggle_select(point)
    
    number_of_types = len(grid.images)
    match_count = process_matches(board, number_of_types, required_matches, refresh)
    handle_score(match_count)
    
    if(match_count == 0):
        swap_items(board, last_point, point)      
        refresh()
        return
        
    # Check if its game over
    moves = test_all(board, required_matches)
    if(len(moves) == 0):
        handle_game_over()
        

def game_over(grid):
    grid.is_overlay_on = True
    grid.overlay_text = "Game Over!"
        

def init_grid(grid, rows, cols, required_matches): 
    
    grid.is_overlay_on = False
    number_of_types = len(grid.images)
    
    is_game_over = True
    count = 0
    
    while is_game_over:
        board = np.random.randint(number_of_types, size=(rows, cols))
        process_matches(board, number_of_types, required_matches, lambda: None)
        moves = test_all(board, required_matches)
        is_game_over = len(moves) == 0
        count = count + 1
        if(count % 100 == 0):
            print(str(count) + str(is_game_over))
        
    grid.data = board    
    

def create_grid(required_matches, handle_score):        
    # Create a new grid and 
    grid = GameGrid()
    grid.on_click(
        lambda g, r, c: handle_click(g, (r, c), required_matches, handle_score, 
                                     lambda: game_over(grid)))        
    return grid

In [4]:
game = new_game(15, 15, 4)
display(game)